In [1]:
import os
from dotenv import load_dotenv
load_dotenv()
api_key = os.getenv("OPEN_API_KEY")

In [2]:
llm_config_mini = {"model": "gpt-4o-mini"}
llm_config_large = {"model": "gpt-4o"}

# Core Agents

In [4]:
from autogen import ConversableAgent

In [18]:
entity_extraction_agent = ConversableAgent(
    name="Entity_Extraction_Agent",
    system_message='''You are an Entity Extraction Agent,
    You are an AI Agent tasked with extracting entitites from SAR Narratives.
    
    Using the SAR narrative shared by the user, do the following:

    Step 1) Extract the main entities described. Entities could be Individual or Organizations or Financial Institutions 

    Step 2) Extract the account IDs described. Account IDs could be numeric or alphanumeric. If an Account ID is missing, create a Dummy account ID with the prefix "Dummy_".e.g. Dummy_001, Dummy_002 and so on

    Step 3) Map the extracted account IDs to Individuals or Organizations. Return this in the form of a dictionary, E.g. {"Entity_Name": ["Account_ID"]}

    Step 4) Map the extracted account IDs to Financial institutions where the account is held. Return this in the form of a dictionary, E.g. {"Financial Instituion": ["Account_ID"]}.
            If the name of the institution is not specified, create a Dummy institution name e.g Dummy_Bank_1, Dummy_Bank_2 etc.

    An example is given below.

    Narrative:

    John deposited $5000 in Cash into Acct #345723 at Bank of America. John sends $3000 to Jill's account at  Chase. Jill deposited $3000 in Cash into her Acct at Chase Bank.
    John and Jill own a business Acme Inc that has a  Business account, Account #98765 . John sends $2000 from Acct #345723 to Account #98765. Jill sends $1000 from 
    her Acct at Chase Bank to Acct #98765.

    Step 1) Extract the main entities described. Identify the Individuals , Organizations and Financial Instituions mentioned. 
    Individuals and Organizations are entities that usually conduct transactions through Financial institutions. Here, John and Jill are Individuals. Acme Inc is an Organization.
    Financial institutions are institutions such as banks or insurance companies that offer financial services to customers. Hence Bank of America and Chase Bank are Financial institutions.

    Record this as a dictionary named Entities.


    Entities = {"Individuals": ["John", "Jill"],"Organizations":["Acme Inc"],"Financial Institutions":["Chase"," Bank of America"]}}

    Step 2) Extract the account IDs described. Accounts mentioned in the narrative are #345723,#98765. There is also an account at Chase Bank that is missing. Assign a dummy account ID 
    (eg. "Dummy_001", "Dummy_002")to this account.

    Record this in a list named Account_IDs
    Account_IDs = ["345723","98765","Dummy_001"]

    Step 3) Map the Account IDs extracted in Step 2 to Financial Instititions where they are held. Account ID #345723 is held at bank of America.Account ID Dummy_001 is held at Chase Bank.
    Account ID #98765 is referenced but the Financial instituion where it is held is not specified. So it can be assumed to be held at "Dummy_Bank_1"

    Record this in a dictionary named FIs_to_Accts.
    FIs_to_Accts = {"Bank of America":["345723"],"Chase Bank":["Dummy_001"],"Dummy_Bank_1":"98765"}

    
    Step 4) Map the extracted Account IDs extracted in Step 2 to Individuals or Organizations that are customers of the Financial Institutions. Account ID #345723 belongs to John. 
    Account ID #98765 belongs to Acme Inc. 
    Account ID Dummy_001 belongs to Jill.
    

    Record this in  a dictionary named Customers_to_Accts

    Customers_to_Accts = {"John":["345723"],"Jill":["Dummy_001"],"Acme Inc":"98765"}
    
    
    ''',
    llm_config=llm_config_large,
    code_execution_config=False,
    human_input_mode="NEVER",
)

[autogen.oai.client: 11-02 12:03:08] {184} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.


In [6]:
entity_resolution_agent = ConversableAgent(
    name="entity_resolution_agent",
    system_message='''
    You are an AI Agent tasked with assigning Customer IDs to entities described in a SAR and mapping Account IDs to these customer IDs.

    You will have the following four pieces of information.

    
    1) List of Account IDs given by the list Account_IDs. </n>
    2) The mapping between Individuals and Organizations to Account IDs  , given by the dictionary Customers_to_Accts. Note, account IDs starting with the prefix "Dummy_" </n>
    3) The mapping between Financial Institutions to Account IDs given by the dictionary FIs_to_Accts. If the Financial institution for certain accounts are not specified, a dummy financial institution
      "Dummy_Bank" will be used for them.</n>
    4) SAR narrative 
    
    Using this information, do the following:

    Step 1)  For each Financial institution in FIs_to_Accts, identify the accounts at that institution. Note, account IDs starting with the prefix "Dummy_" are placeholders for account IDs that
             have not been identified in the narrative.

    Step 2)  Identify the owners of the accounts from  Customers_to_Accts 

    Step 3)  If an Individual is a beneficial owner of an Organization and the Organizations's account, note this in a scratch pad.

    Step 4)  For each Financial institution, map each account ID to a Customer ID (E.g. CUST_001, CUST_002). If multiple accounts are owned by the same Individual or Organization, map them to the same Customer ID

    Return this final mapping between Customer IDs and Account IDs in the form of a JSON

    An example is given below, demarcated by the delimiter ----.

    ----

    1) Account_IDs = ["345723","98765","Dummy_001"]
    
    2) Customers_to_Accts = {"John":["345723"],"Jill":["Dummy_001"],"Acme Inc":"98765"}

    3) FIs_to_Accts =  {"Bank of America":["345723","98765"],"Chase Bank":["Dummy_001"]}

    4) Narrative: </n>
    John deposited $5000 in Cash into Acct #345723 at Bank of America. John sends $3000 to Jill's account at Chase. Jill deposited $3000 in Cash into her Acct at Chase Bank.
    John and Jill own a business Acme Inc that has a  Business account, Account #98765 at Bank of America. John sends $2000 from Acct #345723 to Account #98765. Jill sends $1000 from 
    her Acct at Chase Bank to Acct #98765.

    
    Step 1) For each Financial institution in FIs_to_Accts, identify the accounts at that institution. 

    At Bank of America, there are two accounts #345723 and #98765
    At Chase Bank, there is one account #Dummy_001

    Step 2) Identify the owners of the accounts from Customers_to_Accts 

    Account  #345723 is owned by John
    Account #Dummy_001 is owned by Jill.
    Account #98765 is owned by Acme Inc

    Step 3)If an Individual is a beneficial owner of an Organization and the Organizations's account, note this in a scratch pad. If two individuals are join owners of an account, note this in a scratch pad.

    John and Jill are co-owners of the business Acme Inc and therefore its accounts.

    Step 4) For each Financial institution, map each account ID to a Customer ID (E.g. CUST_001, CUST_002). If multiple accounts are owned by the same Individual or Organizations, map them to the
    same Customer ID

    At Bank of America, the account #345723 is owned by John, the account 98765 is owned by Acme Inc. So these two account IDs can be mapped to two customer IDs. The Account Dummy_001 
    is an account at Chase as per FIs_to_Accts and is owned by Jill as per Customers_to_Accts. This can be assigned to a different customer ID

    {"Bank of America": {"CUST_001": ["345723"], "CUST_002":["98765"]},
    "Chase Bank": {"CUST_003":["Dummy_001"]}}
    
    Return this as a JSON.
    ----



    ''',
    llm_config=llm_config_mini,
    code_execution_config=False,
    human_input_mode="NEVER",
)

[autogen.oai.client: 11-02 11:56:42] {184} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.


In [7]:
narrative_extraction_agent = ConversableAgent(
    name="narrative_extraction_agent",
    system_message='''
    You are an AI Agent tasked with extracting or summarizing  parts of a narrative that describe activity conducted by certain accounts.

    You will have the following three pieces of information.

    1)  SAR narrative

    2)  The mapping between Individuals and Organizations referenced in the narrative to Account IDs which is given by the dictionary Customers_to_Accts

    3) List of Account IDs given by a list Account_IDs
    
    Using the SAR narrative shared by the user, Do the following:

    Step 1) Identify an Account ID from the list of Account IDs.Note that account IDs starting with the prefix "Dummy_" are placeholders for account IDs that have not been explicitly described in the narrative.

    Step 2)  From the dictionary Customers_to_Accts, Identify the  individual and Organization  who owns the account.

    Step 3) Extract or summarize the narrative which describes transactions involving this account.

    An example is given below, demarcated by the delimiter ----.

    ----

    1) SAR Narrative:
        John deposited $5000 in Cash into Acct #345723 at Bank of America. John sends $3000 to Jill's account at Chase. Jill deposited $3000 in Cash into her Acct at Chase Bank.
        John and Jill own a business Acme Inc that has a  Business account, Account #98765 at Bank of America. John sends $2000 from Acct #345723 to Account #98765. Jill sends $1000 from 
        her Acct at Chase Bank to Acct #98765.
    
    2) Customers_to_Accts = {"John":["345723"],"Jill":["Dummy_001"],"Acme Inc":"98765"}

    3) Account_IDs = ["345723","98765","Dummy_001"]

    
    Step 1) The first account ID  is 345723

    Step 2) This Account ID is owned by John.

    Step 3)Extract the part of the narrative that references the Account 345723

    {"345723": "John deposited $5000 in Cash into Acct #345723 at Bank of America. John sends $3000 to Jill's account at Chase.John sends $2000 from Acct #345723 to Account #98765" }

    
    Now repeat the process for the second account ID

    Step 1) The second  account ID  is 98765

    Step 2) This Account ID is owned by Acme, Inc.

    Step 3)Extract the part of the narrative that references the Account 98765

    {"98765": "John sends $2000 from Acct #345723 to Account #98765" }

    Now repeat the process for the third Account ID.

    Step 1) The third  account ID  is Dummy_001. Given the acocunt ID starts with Dummy_ , there won't be direct references to this account ID in the narrative. 

    Step 2) This Account ID is owned by Jill. Given the Account ID is dummy, pay attention to transactions made by Jill,the owner of the dummy account.

    Step 3) Extract the part of the narrative that references the Account Dummy_001

    {"Dummy_001": "John sends $3000 to Jill's account at Chase. Jill deposited $3000 in Cash into her Acct at Chase Bank." }

    Consolidate narratives for all accounts.

     Narrative : {"345723": "John deposited $5000 in Cash into Acct #345723 at Bank of America. John sends $3000 to Jill's account at Chase.John sends $2000 from Acct #345723 to Account #98765",
      "98765": "John sends $2000 from Acct #345723 to Account #98765",
      "Dummy_001": "John sends $3000 to Jill's account at Chase. Jill deposited $3000 in Cash into her Acct at Chase Bank." 
    }


    ----



    ''',
    llm_config=llm_config_large,
    code_execution_config=False,
    human_input_mode="NEVER",
)

[autogen.oai.client: 11-02 11:56:44] {184} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.


# Sequential Chat with Entity Extraction Agent + Reflection with Critic Agent

In [8]:
#Complex message possibly requiring reflection
sar1 = '''
This SAR is being filed to summarize suspicious cash deposits and wire transfer activity conducted by John Doe, account #12345678910.  John  Doe has been a bank customer since April 2000.  Mr. Doe is a college  student and employed part-time at Quickie Car Wash.  

Cash deposits to Mr. Doe’s personal checking account are structured to possibly circumvent federal reporting requirements.  The deposits are  followed by immediate wire transfers to Aussie Bank in Sydney, Australia to a single beneficiary, Jennifer Doe, account #981012345, with an address located in Australia.  Specifically the following activity has been 
observed: cash deposits (dates followed by amounts): 03/15/02  $9,950.00; 03/17/02 $9,700.00; 03/18/02 $10,000; total: $29,650.  Wire transfers out (dates followed by amounts): 03/16/02 $9,900.00, 03/18/02  $9,700.00, 03/19/02 $9,900.00.  

The volume and frequency of the deposits is not consistent with previous banking transactions conducted by Mr. Doe.  The amounts of currency do not appear consistent with the 
customer’s stated employment.  Also, the relationship between the customer and Jennifer Doe and the purpose for the wire activity is unknown. 
Therefore, due to the structured cash deposits by the customer on almost consecutive days into the account, and the immediate wire transfer of the funds out of the account to Jennifer Doe, Aussie bank, account  #981012345, Sydney Australia, this SAR is being filed.  Investigation is continuing. 
The bank’s financial intelligence unit in Big City, FL, maintains all records related to this SAR

'''


message1 = f'''
 Please analyze the following supicious activity report:
 {sar1}

'''

In [9]:
#Simpler message typically not requiring reflection
sar2 = '''
  John deposited $5000 in Cash into Acct #345723 at Bank of America. John sends $3000 to Jill's account, Acct #12345 at Chase. 
'''

message2 = f'''
 Please analyze the following supicious activity report:
 {sar2}

'''

In [10]:
sar_agent = ConversableAgent(
    name="SAR_Agent",
    system_message="You are assigned a SAR from which to extract transactions. You do this by sharing this SAR with other assistant agents who "
                   "work on the SAR and extract useful information.",
    is_termination_msg = lambda x: x.get("content","") and x.get("content", "").find("SATISFACTORY")>=0,
    llm_config=llm_config_mini,
    max_consecutive_auto_reply=5,
    human_input_mode="NEVER",
)

[autogen.oai.client: 11-02 11:56:57] {184} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.


In [11]:
# Reflection Agent for entity extraction
reflection_agent_ee  = ConversableAgent(
    name = "reflection_agent_ee",
    system_message ='''
                        You receive the following information to review:
                            "Account IDs" : List of Account IDs mentioned in the Suspicious Aactivity Repot (SAR)
                            "FIs_to_Accts" : Mapping between above Account IDs and Financial instituions(FI) they belong to. If the name of the FI is not explicitly mentioned, it will
                                             be identified as a Dummy_Bank
                            "Customers_to_Accts": Mapping between customer names and accounts they own.


                        You will review this information for accuracy in the context of the provided SAR and provide feedback if any information is missing or incorrect.
                        If the information is satisfactory, just return the word SATISFACTORY.

                    ''',
    llm_config = llm_config_mini
)

[autogen.oai.client: 11-02 11:57:00] {184} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.


In [12]:
def reflection_message_ee(recipient,sar,sender,config):
    print ("Reflecting...")
    return f''' Review the information below in the context of the SAR narrative provided below and provide any feedback for improvement
                INFORMATION TO REVIEW: \n {recipient.chat_messages_for_summary(sender)[-1]['content']}
                SAR NARATIVE: \n {sar}

            '''

In [13]:

nested_chat_queue = [
    {
        "recipient": reflection_agent_ee,
        "message": reflection_message_ee,
        "summary_method": "reflection_with_llm",
        "summary_args": {"summary_prompt" : 
            "Return review into as JSON object only:"
            "{'Review': < Review >} ",},
        "max_turns": 1,
    },
]

In [14]:
sar_agent.register_nested_chats(
    nested_chat_queue,
    trigger=entity_extraction_agent,
    # position=4,
)

In [15]:

res = sar_agent.initiate_chat(
    recipient=entity_extraction_agent,
    message=sar2,
    max_turns=2,
    summary_method="reflection_with_llm",
    summary_args = {
                 "summary_prompt" : "Return the extracted entity information as JSON object only: "
                                "{'Account_IDs': ['Acct_ID_1','Acct_ID_2','Acct_ID_3']}"
                                "{'FIs_to_Accts': { 'Entity1': ['Acct_ID_1'],'Entity2':['Acct_ID_2','Acct_ID_3]} }"
                                "{'Customers_to_Accts': { 'Entity2': ['Acct_ID_1'],'Entity4':['Acct_ID_2'],'Entity5':['Acct_ID_3']} }" 
    }
)

SAR_Agent (to Entity_Extraction_Agent):


  John deposited $5000 in Cash into Acct #345723 at Bank of America. John sends $3000 to Jill's account, Acct #12345 at Chase. 


--------------------------------------------------------------------------------
Entity_Extraction_Agent (to SAR_Agent):

Step 1) Extract the main entities described.

Entities = {
    "Individuals": ["John", "Jill"],
    "Organizations": [],
    "Financial Institutions": ["Bank of America", "Chase"]
}

Step 2) Extract the account IDs described.

Account_IDs = ["345723", "12345", "Dummy_001"]

Step 3) Map the Account IDs extracted in Step 2 to Financial Institutions where they are held.

FIs_to_Accts = {
    "Bank of America": ["345723"],
    "Chase": ["12345"],
    "Dummy_Bank_1": ["Dummy_001"]
}

Step 4) Map the extracted Account IDs to Individuals or Organizations that are customers of the Financial Institutions.

Customers_to_Accts = {
    "John": ["345723"],
    "Jill": ["12345"],
    "Dummy_Organization_1": ["D

In [17]:
print(res.summary)

```json
{
  "Account_IDs": ["345723", "12345"],
  "FIs_to_Accts": {
    "Bank of America": ["345723"],
    "Chase": ["12345"]
  },
  "Customers_to_Accts": {
    "John": ["345723"],
    "Jill": ["12345"]
  }
}
```


### Try Entity Extraction Agent with GPT-4o

In [19]:
res = sar_agent.initiate_chat(
    recipient=entity_extraction_agent,
    message=sar2,
    max_turns=2,
    summary_method="reflection_with_llm",
    summary_args = {
                 "summary_prompt" : "Return the extracted entity information as JSON object only: "
                                "{'Account_IDs': ['Acct_ID_1','Acct_ID_2','Acct_ID_3']}"
                                "{'FIs_to_Accts': { 'Entity1': ['Acct_ID_1'],'Entity2':['Acct_ID_2','Acct_ID_3]} }"
                                "{'Customers_to_Accts': { 'Entity2': ['Acct_ID_1'],'Entity4':['Acct_ID_2'],'Entity5':['Acct_ID_3']} }" 
    }
)

SAR_Agent (to Entity_Extraction_Agent):


  John deposited $5000 in Cash into Acct #345723 at Bank of America. John sends $3000 to Jill's account, Acct #12345 at Chase. 


--------------------------------------------------------------------------------
Entity_Extraction_Agent (to SAR_Agent):

Step 1) Extract the main entities described.

Entities = {
    "Individuals": ["John", "Jill"],
    "Organizations": [],
    "Financial Institutions": ["Bank of America", "Chase"]
}

Step 2) Extract the account IDs described.

Account_IDs = ["345723", "12345"]

Step 3) Map the extracted account IDs to Individuals or Organizations.

Customers_to_Accts = {
    "John": ["345723"],
    "Jill": ["12345"]
}

Step 4) Map the extracted account IDs to Financial institutions where the account is held.

FIs_to_Accts = {
    "Bank of America": ["345723"],
    "Chase": ["12345"]
}

--------------------------------------------------------------------------------
SAR_Agent (to Entity_Extraction_Agent):

### Extr

In [21]:
print(res.summary)

```json
{
  "Account_IDs": ["345723", "12345"]
}
{
  "FIs_to_Accts": {
    "Bank of America": ["345723"],
    "Chase": ["12345"]
  }
}
{
  "Customers_to_Accts": {
    "John": ["345723"],
    "Jill": ["12345"]
  }
}
```


# Sequential Chat with Entity Resolution Agent + Reflection with Critic Agent